<a href="https://colab.research.google.com/github/hugomedsa/valuation_acoes_b3/blob/main/Passo_1_Baixar_c%C3%B3digos_CVM_%2B_tickers_B3_%2B_setores_e_segmentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurações iniciais

In [ ]:
!pip install progress -q

  Preparing metadata (setup.py) ... done


In [ ]:
from bs4 import BeautifulSoup
from google.colab import drive
from multiprocessing.pool import ThreadPool
import pandas as pd
import re, ssl
from time import sleep
from tqdm import tqdm
from urllib.request import urlopen,Request
from google.colab.data_table import DataTable

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}
def sopa(url):
  context = ssl._create_unverified_context()
  req = Request(url,headers=headers)
  response = urlopen(req,context = context)
  # html = response.read().decode('ISO-8859-1')
  # html = " ".join(html.split()).replace('> <', '><')
  soup = BeautifulSoup(response)#,'html.parser')
  return soup

# Localizando os tickets da B3

In [ ]:
url = 'https://www.fundamentus.com.br/resultado.php?'
soup = sopa(url)
soup_tickers = soup.findAll('span',{'class':'tips'})
tickers = [ticker.text for ticker in soup_tickers]
len(tickers)

986

#Encontrando os setores e segmentos

In [ ]:
def add_caracteristicas(categoria,nome):
  valor = soup.find_all('a',{'href': re.compile(rf'resultado\.php\?{nome}\=\d+')})
  valore =''
  for se in valor: #Setor ou Segmento
    valore +=f'{se.text}'
  if valore == '':
    categoria.append('-')
  else:
    categoria.append(valore)

In [ ]:
tickets = ['']
setores = []
segmentos = []

with tqdm(total=len(tickers),) as pbar:
  for tick in sorted(tickers):
    if tick[:4] not in tickets[-1]:
      tickets.append(tick)

      url = f'https://www.fundamentus.com.br/detalhes.php?papel={tick}'
      soup = sopa(url)
      add_caracteristicas(setores,'setor')
      add_caracteristicas(segmentos,'segmento')
    else:
      tickets.append(tick)
      setores.append(setores[-1])
      segmentos.append(segmentos[-1])
    pbar.update()

segmentos_setores_b3 = pd.DataFrame({'ticker':tickets[1:],'setor':setores,'segmento':segmentos})
segmentos_setores_b3.to_excel('/content/drive/MyDrive/segmentos_setores_b3.xlsx')

100%|██████████| 986/986 [03:02<00:00,  5.41it/s]


In [ ]:
segmentos_setores_b3

,ticker,setor,segmento
0,AALR3,Serv.Méd.Hospit. Análises e Diagnósticos,Serv.Méd.Hospit. Análises e Diagnósticos
1,ABCB3,Intermediários Financeiros,Bancos
2,ABCB4,Intermediários Financeiros,Bancos
3,ABEV3,Bebidas,Cervejas e Refrigerantes
4,ABYA3,Construção Civil,Incorporações
...,...,...,...
981,WLMM4,Comércio,Material de Transporte
982,WMBY3,"Tecidos, Vestuário e Calçados",Fios e Tecidos
983,WSON33,Transporte,Serviços de Apoio e Armazenagem
984,YDUQ3,Diversos,Serviços Educacionais


#Pesquisando os códigos CVM

In [ ]:
lista = 'A B C D E F G H I J K L M N O P Q R S T U V W Y X Z 0 1 2 3 4 5 6 7 8 9'.split()
grupos = []
with tqdm(total=100) as pbar:
  for letra in lista:
    url = 'https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CiaAb/FormBuscaCiaAbOrdAlf.aspx?LetraInicial='+letra
    soup = sopa(url)
    registros = soup.find_all('a',{'class':'HRefPreto'})
    linha = []
    cont = 1
    for c in registros:
      linha.append(c.text)
      if cont%5==0:
        if 'Cancelado' not in linha[4]:
          grupos.append([linha[1],linha[3]])
        linha = []
      else:
        pass
      cont+=1
      pbar.update()
len(grupos)

23250it [02:28, 156.84it/s]


732

In [ ]:
pd.DataFrame(grupos, columns=['Nome','CD_CVM']).set_index('CD_CVM').to_excel('/content/drive/MyDrive/empresas_codigos_cvm.xlsx')
groups = pd.read_excel('/content/drive/MyDrive/empresas_codigos_cvm.xlsx')

In [ ]:
def pag_cvm(letra):
  url = 'https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CiaAb/FormBuscaCiaAbOrdAlf.aspx?LetraInicial='+letra
  soup = sopa(url)
  registros = soup.find_all('a',{'class':'HRefPreto'})
  linha = []
  cont = 1
  for c in registros:
    linha.append(c.text)
    if cont%5==0:
      if 'Cancelado' not in linha[4]:
        grupos.append([linha[1],linha[3]])
      linha = []
    else:
      pass
    cont+=1

lista = 'A B C D E F G H I J K L M N O P Q R S T U V W Y X Z 0 1 2 3 4 5 6 7 8 9'.split()

grupos = []

with tqdm(total=100) as pbar:
  with ThreadPool(18) as pool:
    for soup in pool.map(pag_cvm, lista):
      pbar.update()

len(grupos)


 36%|███▌      | 36/100 [01:24<02:30,  2.35s/it] 


732

In [ ]:
pd.DataFrame(grupos, columns=['Nome','CD_CVM']).set_index('CD_CVM').to_excel('/content/drive/MyDrive/empresas_codigos_cvm.xlsx')
groups = pd.read_excel('/content/drive/MyDrive/empresas_codigos_cvm.xlsx')

#Unindo códigos CVM e códigos B3

In [ ]:
b3cvm =  {'ticker':[],'CD_CVM':[]}
def sopaurl(cvm):
  url2 = f'https://bvmf.bmfbovespa.com.br/pt-br/mercados/acoes/empresas/ExecutaAcaoConsultaInfoEmp.asp?CodCVM={cvm}&ViewDoc=1&AnoDoc=2019&VersaoDoc=1&NumSeqDoc=82375'
  print(url2)
  return sopa(url2),cvm
with ThreadPool(50) as pool:
  for soup in pool.map(sopaurl, groups['CD_CVM'].values.tolist()):
    b3_html = soup[0].findAll('a',{'class':'LinkCodNeg'})
    if len(b3_html) > 0:
      b3 = set([c.text for c in b3_html])
      for tic in b3:
        if tic in tickers:
          db = [tic,str(soup[1])]
          if db[1] not in b3cvm:
            b3cvm['ticker'].append(db[0])
            b3cvm['CD_CVM'].append(db[1])

pd.DataFrame(b3cvm).to_excel('/content/drive/MyDrive/codigos_cvm_b3.xlsx')
codb3cvm = pd.DataFrame(b3cvm).dropna()

In [ ]:
codb3cvm

,ticker,CD_CVM
0,QUAL3,22497
1,CTKA4,4146
2,CTKA3,4146
3,KEPL3,7870
4,KLBN11,12653
...,...,...
471,TRPL4,18376
472,TRPL3,18376
473,CURY3,25100
474,CVCB3,23310


In [ ]:
codigos_e_setores = pd.merge(segmentos_setores_b3, codb3cvm, how='left', on='ticker')

In [ ]:
pd.DataFrame(codigos_e_setores).to_csv('/content/drive/MyDrive/codigos.csv')